Files to update:
- Ballade pour Adeline (with fingering) - Senneville
- Czerny - School Of Velocity - Op. 299 No. 7
- Fantasia_in_C_Minor_BWV_906__Johann_Sebastian_Bach_includes_fingering

In [ ]:
import music21
import random
import os

from music21 import *

import pandas as pd

music_score_path = 'C:/Program Files/MuseScore 4/bin/MuseScore4.exe'
music21.environment.set('musescoreDirectPNGPath', music_score_path)


In [ ]:
!mkdir output_new_left
!mkdir output_new_right
!mkdir new_csv_train

In [ ]:
# !yes | add-apt-repository ppa:mscore-ubuntu/mscore3-stable &> /dev/null
# !apt update &> /dev/null
# !apt install musescore3 &> /dev/null
# print("MuseScore installation finished")


yes: standard output: Broken pipe
MuseScore installation finished


In [ ]:
def Stream2PIG(sf, fname, test=False, n=0, beam=0, time_unit=0.5):
    """
    Convert a Stream to PIG CSV format using Pandas.
    
    sf: music21 Stream object.
    fname: Output CSV file name.
    beam: 0 for right hand, 1 for left hand.
    time_unit: Time scaling factor.
    """
    data = []
    idx = 0
    name = None
    
    for measure in sf.getElementsByClass(stream.Measure):
        measure_idx = measure.measureNumber if measure.measureNumber is not None else idx
        
        for elem in measure.notesAndRests:
            onset = elem.offset * time_unit
            offset = (elem.offset + elem.quarterLength) * time_unit
            
            if isinstance(elem, note.Note):
                name = elem.nameWithOctave.replace('-', 'b')
                fingering = None
                if elem.articulations:
                    for art in elem.articulations:
                        if isinstance(art, articulations.Fingering):
                            fingering = art.fingerNumber
                
                
                if fingering is not None and (fingering < 1 or fingering > 5):
                    print(f"NOTE: {name}, Fingering {fingering} outside the valid range (1-5) at measure (idx): {measure_idx}, .")
                    measure.show()  

                data.append([idx, round(onset, 3), round(offset, 3), name, 0, 0, beam, fingering if fingering is not None else 99])
                idx += 1
            
            elif isinstance(elem, chord.Chord):
                name_pitch = [pitch.nameWithOctave.replace('-', 'b') for pitch in elem.pitches]
                fingering = []
                
                for art in elem.articulations:
                    if isinstance(art, articulations.Fingering):
                        finger = art.fingerNumber if art.fingerNumber is not None else 99
                        fingering.append(finger)
                
                while len(fingering) < len(name_pitch):
                    fingering.append(99)
                
                for i in range(len(name_pitch)):
                    if int(fingering[i]) < 1 or int(fingering[i]) > 5:
                        print(f"NOETE: {name_pitch[i]}, Fingering {fingering[i]} outside the valid range (1-5) at measure (idx): {measure_idx}.")
                        measure.show()  
                        
                    data.append([idx, round(onset, 3), round(offset, 3), name_pitch[i], 0, 0, beam, fingering[i]])
                    idx += 1
    
    df = pd.DataFrame(data, columns=["idx", "onset", "offset", "name", "col1", "col2", "beam", "fingering"])
    
    df.to_csv(fname, index=False, header=False)
    return df


In [ ]:
def xml_to_csv(input_file, pig_right, pig_left=""):
    
    piece = music21.converter.parse(input_file)
    right_hand = piece.parts[0]
    # left_hand = bwv295.parts[1]
    
    Stream2PIG(right_hand, pig_right, test=True, n=11, beam=0, time_unit=1.0)
    header = ["ID", "onset_time", "offset_time", "pitch", \
          "onset_vel", "offset_vel", "channel", "finger"]
    df = pd.read_csv(pig_right, sep=" ", names=header)
    a = df[df["finger"] == '99']
    if len(a) > 0:
        print(a)
        raise ValueError(f"Found 99 in {input_file}")
     


In [ ]:
for dirname, _, filenames in os.walk('./processed_xml/mxl/'):
    for filename in filenames:
            full_name = os.path.join(dirname, filename)
            print(filename, end='\n\n')
            xml_to_csv(full_name, f'output_new_right/right_{filename[:-4]}.csv',  
                        f'output_new_left/left_{filename[:-4]}.csv')




50 Little Etudes, No. 21 - 30 – Isaak Berkovich.mxl

50_Little_Etudes_No._11_-_20__Isaak_Berkovich.mxl

50_Little_Etudes_No._1_-_10__Isaak_Berkovich.mxl

Air_on_the_G_string_with_fingering.mxl

All of Me - Lead Sheet with Fingering.mxl

Bach G minor arr. Luo Ni (with fingering).mxl

Blumenlied, Op.39 – Gustav Lange (with fingerings).mxl

Canon_in_D_By_OMID.mxl

Chopin_-_Nocturne_No.20_with_fingering_in_C_Minor_Op._posth.mxl

Cuphead - Inkwell Isle (Beginner-Intermediate with Fingering) – Kristofer Maddigan.mxl

Czerny - Op 740_ The Art of Finger Dexterity - No 11.mxl

Czerny - School Of Velocity - Op. 299 No. 11.mxl

Czerny - School Of Velocity - Op. 299 No. 2.mxl

Czerny - School Of Velocity - Op. 299 No. 3.mxl

Czerny - School Of Velocity - Op. 299 No. 4.mxl

Czerny - School Of Velocity - Op. 299 No. 5.mxl

Czerny - School Of Velocity - Op. 299 No. 6.mxl

Czerny - School Of Velocity - Op. 299 No. 8.mxl

Czerny - School Of Velocity - Op. 299 No. 9.mxl

Czerny_-_School_Of_Velocity_-_Op